In [18]:
import pandas as pd
import numpy as np


# Load  RE288 table from Excel, table should have columns: ['outs', 'bases', 'count', 'RE']
file_path = "RED288.xlsx"
xls = pd.ExcelFile(file_path)

# Read all sheets into a dictionary of DataFrames
RE288 = {sheet_name: pd.read_excel(xls, sheet_name=sheet_name,index_col=0) for sheet_name in xls.sheet_names}

# --- Helper to get RE given state ---
def get_re(outs, bases, balls,strikes):
    if outs==0:
        data=RE288['0_outs']
    elif outs==1:
        data=RE288['1_out']
    elif outs==2:
        data=RE288['2_outs']
        
    def display_combination(input_str):
    # Initialize a set to track which numbers are present
        present = set()
        
        # Check each character in the input string
        for char in input_str:
            if char in {'1', '2', '3'}:
                present.add(char)
        
        # Build the output string
        output = []
        for num in ['1', '2', '3']:
            if num in present:
                output.append(num)
            else:
                output.append('_')
        
        # Join with spaces and return
        return ' '.join(output)
    
    count=f'[{balls}-{strikes}]'
    bases_str=display_combination(bases)
    
    red_val=data.loc[bases_str,count]

    return red_val

num_pitches=round(.1081*292) #from ABS analysis we found about 10.8% of pitches are marginal, and there are 292 pitches a game on average

def simulate_game(n_pitches=num_pitches, challenges=2):
    
    total_value = 0.0
    remaining_challenges = challenges

    for i in range(1, n_pitches+1):
        q = 0.5  # 50% chance of overturn

        # Example state (replace with actual game logic per pitch)
        outs = np.random.choice([0,1,2])
        bases = np.random.choice(['1', '2', '3', '12', '13', '23', '123', ''])
        balls = np.random.randint(0,4)
        strikes = np.random.randint(0,3)

        # Get actual RED from your table (includes leverage)
        RED = get_re(outs, bases, balls, strikes)

        # Expected value
        exp_value = q * RED
    
        
        frac = i / n_pitches  # fraction of marginal pitches elapsed
        
        a = 1.0       # initial scale for exponential
        b = 3.0       # decay rate (adjust as needed)
        
        theta = min(1.75, a * np.exp(-b * frac))

        if challenges == 1:
            theta += 0.003
        
        
        # Policy decides
        if remaining_challenges > 0 and (exp_value>theta):
            if np.random.rand() < q:  # Successful challenge
                total_value += RED 
                # Challenge is renewable → keep count
            else:
                remaining_challenges -= 1

    return total_value



# --- Run many simulations to estimate performance ---
def evaluate(n_games=50000):
    results = [simulate_game() for _ in range(n_games)]
    return np.mean(results), np.percentile(results,[5,50,95])

avg, dist = evaluate()
print("Avg value captured:", avg)
print("5/50/95 percentiles:", dist)


Avg value captured: 0.6753292
5/50/95 percentiles: [0.   0.43 2.25]
